In [1]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
from pyincore.analyses.buildingfunctionality import BuildingFunctionality
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.epfdamage import EpfDamage

### Define client

In [2]:
client = IncoreClient()

### Chaining with Joplin Building Damage

In [3]:
bldg_dmg = BuildingDamage(client)

hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"  # The simulated EF-5 tornado shows geographical locations and the range
                                        # of wind speed of tornado hazard in Joplin.

mapping_id = "5d8942dfb9219c068ea795ea"  # Create a mapping to assign tornado fragilities to 19 building archetypes.
fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

bldg_dataset_id = "5dbc8478b9219c06dd242c0d"  # Load the Joplin building dataset, which is a package of GIS files.

bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
result_name = "Joplin_bldg_dmg_result"
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)  # Define the result name, etc., and choose the number of CPU locally
                                      # to run the model parallelly.
bldg_dmg.run_analysis()  # Run the building damage module to get building damage results for Joplin in a .csv file format.
                         # The building damage results herein are referred to fragilities at three damage states (moderate,
                         # heavy, complete) for 28152 buildings located in Joplin that fall within the tornado path or not.

True

In [4]:
building_dmg_result = bldg_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", building_dmg_result)
mc.set_parameter("result_name", "Joplin_mcs_buildings_samples")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["insignific", "moderate", "heavy", "complete"])
mc.set_parameter("failure_state_keys", ["moderate", "heavy", "complete"])
mc.run_analysis()

True

In [5]:
print(mc.get_output_datasets())
building_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
building_damage_mcs_samples.get_dataframe_from_csv()

{'failure_probability': <pyincore.dataset.Dataset object at 0x7f8ef17a1cd0>, 'sample_failure_state': <pyincore.dataset.Dataset object at 0x7f8ef17a1fd0>}


guid              failure
0      b497b88f-a4ba-48b1-8438-5cfc68147cfc  1,1,1,1,1,1,1,1,1,1
1      6eadcc3e-7bb5-463a-aca8-f1d16f8ac7cc  1,1,1,1,1,1,1,1,1,1
2      758fb3de-108a-4e04-895e-fdc4b8627876  1,1,1,1,1,1,1,1,1,1
3      7b706697-bf9b-4883-90f6-862d67a8e64b  1,1,1,1,1,1,1,1,1,1
4      19c91f9b-f679-4263-b7a3-e76fd859c4b1  1,1,1,1,1,1,1,1,1,1
...                                     ...                  ...
28147  f997af99-0124-4319-8bde-57bd4b1f521b  1,1,1,1,1,1,1,1,1,1
28148  0d31cb6b-c16b-4ac8-ade3-a507a17a4a35  1,1,1,1,1,1,1,1,1,1
28149  e8e4e182-dbd0-4464-9b0f-841f4f775c15  1,1,1,1,1,1,1,1,1,1
28150  22dae85e-f4c0-4aaf-adf1-e92b769ca72d  1,1,1,1,1,1,1,1,1,1
28151  3ee4715c-d253-4f10-ba18-d680e5364eb3  1,1,1,1,1,1,1,1,1,1

[28152 rows x 2 columns]

In [6]:
client.clear_cache()

### Chaining with Joplin EPF damage

#### Electrical Power Facility - Substations Damage

In [7]:
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"  # The simulated EF-5 tornado shows geographical locations and the range
                                        # of wind speed of tornado hazard in Joplin.
epf_substations_id = "5d92355bb9219c06ae7e386a"

epf_sub_mapping_id = "5d8a326fb9219c068ea798e7"  # Create a mapping to assign tornado fragilities to substations.

epf_substations_dmg = EpfDamage(client)

epf_substations_dmg.load_remote_input_dataset("epfs", epf_substations_id)
                               # Load the Joplin substations dataset, which is a package of GIS files.

fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(epf_sub_mapping_id))
epf_substations_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

result_name = "Joplin_epf_substations_dmg_result"
epf_substations_dmg.set_parameter("result_name", result_name)
epf_substations_dmg.set_parameter("hazard_type", hazard_type)
epf_substations_dmg.set_parameter("hazard_id", hazard_id)
epf_substations_dmg.set_parameter("num_cpu", 1)
epf_substations_dmg.set_parameter("fragility_key", "substations")  # Define the result name, etc., and choose the number
                                                       # of CPU locally to run the model parallelly.

epf_substations_dmg.run_analysis()  # Run the EPF damage module to get substations damage results for Joplin in a .csv file format. The
                        # substations damage results herein are referred to fragilities at four damage states (insignificant,
                        # moderate, extensive, complete) for 18 substations located in Joplin that fall within the tornado
                        # path or not

True

In [8]:
substation_dmg_result = epf_substations_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", substation_dmg_result)
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["ds-none","ds-slight","ds-moderat","ds-extensi","ds-complet"])
mc.set_parameter("failure_state_keys", ["ds-slight", "ds-moderat", "ds-extensi", "ds-complet"])

mc.set_parameter("result_name", "Joplin_mcs_substations_samples")
mc.run_analysis()

True

In [9]:
print(mc.get_output_datasets())
substations_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
substations_damage_mcs_samples.get_dataframe_from_csv()

{'failure_probability': <pyincore.dataset.Dataset object at 0x7f8ee82a4710>, 'sample_failure_state': <pyincore.dataset.Dataset object at 0x7f8f4eccdf10>}


guid              failure
0   7f426763-81c6-4cac-b98d-c37a5e439b90  1,1,1,1,1,1,1,1,1,1
1   fc8add00-6760-4752-b1fb-7d6ca813f535  1,1,1,1,1,1,1,1,1,1
2   e8f4939d-763f-4743-bc61-c23e0b8e0f7a  1,1,1,1,1,1,1,1,1,1
3   2cab48ff-c5ce-495e-9662-67c9a177efc4  1,1,1,1,1,1,1,1,1,1
4   eb4ce929-5164-4ebf-ace2-708096d89db5  1,1,1,1,1,1,1,1,1,1
5   4dfb1c7e-c242-4535-bfb4-a706a5151532  1,1,1,1,1,1,1,1,1,1
6   07c488e8-85bd-4dc2-85a2-de85f37723cd  1,1,1,1,1,1,1,1,1,1
7   aa483bec-86e4-4e4b-8ea1-76147877120d  0,0,0,0,0,0,0,0,0,0
8   2f680397-8c75-49ce-b444-871a67c13d34  1,1,1,1,1,1,1,1,1,1
9   12886301-c029-48eb-8781-48c77f026a4f  0,0,0,0,1,0,1,0,1,0
10  d3c076ee-1d8e-4ef8-9856-61be21e0497c  1,1,1,1,1,1,1,1,1,1
11  22eb35e2-0d83-4b00-9121-73765b085fa6  1,1,1,1,1,1,1,1,1,1
12  7bf6efb8-176e-40f4-9441-d3ae01798b8e  1,1,1,1,1,1,1,1,1,1
13  c5e63fe3-de41-46d6-9ff1-060843294e7d  0,0,0,0,0,0,0,0,0,0
14  d37d2ec3-3a8a-4564-a858-90a99ae423d3  1,1,1,1,1,1,1,1,1,1
15  ebdd088e-0eeb-4a4d-a502-0ab2ae5b3a3e  1,1,1,1,1,1,1,1,1,1
16  d0d52400-f069-4091-9fcf-338619cfc1dc  1,1,1,1,1,1,1,1,1,1
17  e936a2a7-5005-4baf-8e33-b8f87e86017c  1,1,1,1,1,1,1,1,1,1

####  Electrical Power Facility - Poles Damage

In [11]:
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"  # The simulated EF-5 tornado shows geographical locations and the range
                                        # of wind speed of tornado hazard in Joplin.
epf_poles_id = "5d923daab9219c06ae84afb0"  # Load the Joplin poles dataset, which is a package of GIS files.

epf_poles_mapping_id = "5d923ea8b9219c06aec7569a"  # Create a mapping to assign tornado fragilities to poles.

epf_poles_dmg = EpfDamage(client)

epf_poles_dmg.load_remote_input_dataset("epfs", epf_poles_id)

fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(epf_poles_mapping_id))
epf_poles_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

result_name = "Joplin_epf_poles_dmg_result"
epf_poles_dmg.set_parameter("result_name", result_name)
epf_poles_dmg.set_parameter("hazard_type", hazard_type)
epf_poles_dmg.set_parameter("hazard_id", hazard_id)
epf_poles_dmg.set_parameter("num_cpu", 4)
epf_poles_dmg.set_parameter("fragility_key", "poles")  # Define the result name, etc., and choose the number
                                                 # of CPU locally to run the model parallelly.
epf_poles_dmg.run_analysis()  # Run the EPF damage module to get substations damage results for Joplin in a .csv file format. The
                        # substations damage results herein are referred to fragilities at four damage states (insignificant,
                        # moderate, extensive, complete) for 18 substations located in Joplin that fall within the tornado
                        # path or not

True

In [12]:
pole_dmg_result = epf_poles_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", pole_dmg_result)
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["ds-none","ds-slight","ds-moderat","ds-extensi","ds-complet"])
mc.set_parameter("failure_state_keys", ["ds-slight", "ds-moderat", "ds-extensi", "ds-complet"])

mc.set_parameter("result_name", "Joplin_mcs_poles_samples")
mc.run_analysis()

True

In [13]:
print(mc.get_output_datasets())
poles_damage_mcs_samples = mc.get_output_dataset('sample_failure_state')
poles_damage_mcs_samples.get_dataframe_from_csv()

{'failure_probability': <pyincore.dataset.Dataset object at 0x7f8f1a5b67d0>, 'sample_failure_state': <pyincore.dataset.Dataset object at 0x7f8f1a5b6990>}


guid              failure
0      cffe8f2e-9887-456b-9625-5ed122b68c45  1,1,1,1,1,1,1,1,1,1
1      54efa7ba-475f-4b1d-8821-42d156cb3f7d  1,1,1,1,1,1,1,1,1,1
2      d0f52ad0-1187-4c0d-9a72-c02687c19778  1,1,1,1,1,1,1,1,1,1
3      901ae6cb-01d5-4936-a3fe-729d930e76ab  1,1,1,1,1,1,1,1,1,1
4      958d0146-1dd5-4b10-8d16-9ec44e7b3010  1,1,1,1,1,1,1,1,1,1
...                                     ...                  ...
23852  c34213db-82d0-4f74-bdd4-31b0ba5b4d5c  1,1,1,1,1,1,1,1,1,1
23853  b576679e-c96d-4060-ba78-9fae2b072c4e  1,1,1,1,1,1,1,1,1,1
23854  1ebc2d81-3b41-40a4-afcd-412dee400d28  1,1,1,1,1,1,1,1,1,1
23855  d43f0edd-d3d6-49cd-a5a0-99cf0adcb395  1,1,1,1,1,1,1,1,1,1
23856  8e4a1c95-12e1-4cd3-8308-4f1c401fbb6b  1,1,1,1,1,1,1,1,1,1

[23857 rows x 2 columns]

In [14]:
client.clear_cache()

## Chaining with building functionality analysis

In [15]:
bldg_func = BuildingFunctionality(client)

# load datasets remotely
# bldg_func.load_remote_input_dataset("building_damage_mcs_samples", "5f0f712433b2700c11f714a0")
# bldg_func.load_remote_input_dataset("substations_damage_mcs_samples", "5f0f71790686cc2aecd3d438")
# bldg_func.load_remote_input_dataset("poles_damage_mcs_samples", "5f0f726c53587c306eca98c5")
    
bldg_func.set_input_dataset("building_damage_mcs_samples", building_damage_mcs_samples)
bldg_func.set_input_dataset("substations_damage_mcs_samples", substations_damage_mcs_samples)
bldg_func.set_input_dataset("poles_damage_mcs_samples", poles_damage_mcs_samples)
bldg_func.load_remote_input_dataset("interdependency_dictionary", "5dcf4a34b9219ca5e4118312")

bldg_func.set_parameter("result_name", "Joplin_mcs_functionality_probability")

True

In [16]:
print(substations_damage_mcs_samples)

In [17]:
bldg_func.run_analysis()

True

In [18]:
print(bldg_func.get_output_datasets())
bldg_functionality = bldg_func.get_output_dataset('result')
bldg_functionality.get_dataframe_from_csv()

{'result': <pyincore.dataset.Dataset object at 0x7f8ee82a4c10>}


building_guid  probability
0      b497b88f-a4ba-48b1-8438-5cfc68147cfc          1.0
1      6eadcc3e-7bb5-463a-aca8-f1d16f8ac7cc          1.0
2      758fb3de-108a-4e04-895e-fdc4b8627876          1.0
3      7b706697-bf9b-4883-90f6-862d67a8e64b          1.0
4      19c91f9b-f679-4263-b7a3-e76fd859c4b1          1.0
...                                     ...          ...
28147  f997af99-0124-4319-8bde-57bd4b1f521b          1.0
28148  0d31cb6b-c16b-4ac8-ade3-a507a17a4a35          1.0
28149  e8e4e182-dbd0-4464-9b0f-841f4f775c15          1.0
28150  22dae85e-f4c0-4aaf-adf1-e92b769ca72d          1.0
28151  3ee4715c-d253-4f10-ba18-d680e5364eb3          1.0

[28152 rows x 2 columns]